# BTTH06: Ẩn dữ liệu khả nghịch trên ảnh bằng phương pháp dịch histogram

TODO: Võ Phương Hòa - 1412192

---

## 1. Cách làm bài và nộp bài

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm (trong đó, `TODO` đầu tiên là bạn phải ghi họ tên và MSSV vào phần đầu của file). Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Mục tiêu chính ở đây là học, học một cách chân thật; nếu bạn được 5 điểm nhưng bạn làm bài một cách chân thật thì vẫn tốt hơn nhiều so với 10 điểm mà không chân thật. Bạn có thể thảo luận với các bạn khác; nhưng bài làm phải là của chính bạn, dựa trên sự hiểu của chính bạn, lời văn là của chính bạn, code là của chính bạn. Bạn không nên đọc bài làm của các bạn năm trước. Nếu bị phát hiện gian lận thì bạn sẽ bị 0 điểm cho toàn bộ phần thực hành.*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart & Run All` để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart & Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt: file `Ex06-ReversibleDataHiding-Img-Hist.ipynb`, và các file dữ liệu mà mình đính kèm (`cover.bmp`, `msg.txt`); rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

## 2. Import

In [1]:
%matplotlib inline
import numpy as np
import bitarray
from scipy.misc import imread, imsave
import matplotlib.pyplot as plt
# You can also import other things ...

## 3. Hàm nhúng

In [2]:
def embed_hist(cover_img_file, msg_file, stego_img_file):
    """
    Embeds a message into a cover image using the hist shift method.
    
    Parameters
    ----------
    cover_img_file : string
        The name of the cover img file.
    msg_file : string
        The name of the message file.
    stego_img_file : string
        The name of the stego img file.
    
    Returns 
    -------
    peak_val : int
        The peak value.
    zero_val : int
        The zero value.
    
    If it's impossible to embed, just return (-1, -1).
    """
    # TODO
    # 1. Read cover img file and process
    # 1.1. Read cover img file
    cover_img = imread(cover_img_file)
    
    # 1.2. Compute cover img hist
    cover_img_hist, _bins = np.histogram(cover_img, bins = range(257))
        
    # 1.3. Compute peak and zero points
    peak_val = np.argmax(cover_img_hist)
    zero_val = np.argmin(cover_img_hist)
    
    # 2. Read msg file and process
    # 2.1. Read msg file
    f = open(msg_file, 'r')
    msg_text = f.read()
    f.close()
    
    # 2.2 Convert msg to msg bits
    msg_bits = bitarray.bitarray()
    msg_bits.frombytes(msg_text)
     
    # 2.3. Check if it is possible to embed
    # (1) zero point exists?
    # (2) len(msg_bits) < cover_img_hist[peak_val]
    if (cover_img_hist[zero_val] > 0) or (len(msg_bits) >= cover_img_hist[peak_val]):
        return (-1, -1)
    
    # 2.4. Add '100...' to msg bits
    msg_bits.extend('1' + '0' * (cover_img_hist[peak_val] - len(msg_bits) - 1))
    
    # 3. Embed msg bits into cover img
    stego_img = cover_img
    below_val = min(peak_val, zero_val)
    upper_val = max(peak_val, zero_val)
    shift_direction = np.sign(zero_val - peak_val)
    mb_idx = 0
    for r in range(stego_img.shape[0]):
        for c in range(stego_img.shape[1]):
            if stego_img[r, c] == peak_val:
                stego_img[r, c] += msg_bits[mb_idx] * shift_direction
                mb_idx += 1
            elif stego_img[r, c] > below_val and stego_img[r, c] < upper_val:
                stego_img[r, c] += shift_direction
    
    # 4. Write stego img to file
    imsave(stego_img_file, np.asarray(stego_img).astype('uint8'))
    
    return (peak_val, zero_val)

## 4. Hàm rút trích

In [3]:
def extract_hist(stego_img_file, peak_val, zero_val, extracted_msg_file, recovered_cover_img_file):
    """
    Extracts the message from a stego image using the hist shift method.
    
    Parameters
    ----------
    stego_img_file : string
        The name of the stego img file.
    peak_val : int
        The peak value used to embed.
    zero_val : int
        The zero value used to embed.
    extracted_msg_file : string
        The name of the extracted message file.
    recovered_cover_img_file : string
        The name of the recovered cover img file.
    """
    # TODO
    # 1. Read stego img file and process
    # 1.1. Read stego img file
    stego_img = imread(stego_img_file)
    
    # 1.2. Compute stego img hist
    stego_img_hist, _bins = np.histogram(stego_img, bins = range(257))
    
    # 2. Extracted msg str (string of bits of msg file) and recovered cover img from stego img
    recovered_cover_img = stego_img
    extracted_msg_str = ''
    below_val = min(peak_val, zero_val)
    upper_val = max(peak_val, zero_val)
    shift_direction = np.sign(zero_val - peak_val)
    below_val += shift_direction
    upper_val += shift_direction
    
    for r in range(recovered_cover_img.shape[0]): # we can understand, 0 <= r,c < 512
        for c in range(recovered_cover_img.shape[1]):
            if (recovered_cover_img[r, c] == peak_val) or (recovered_cover_img[r, c] == peak_val + shift_direction):
                extracted_msg_str += str(abs(recovered_cover_img[r, c] - peak_val)) # '0' or '1'
                recovered_cover_img[r, c] = peak_val
            elif recovered_cover_img[r, c] > below_val and recovered_cover_img[r, c] < upper_val:
                recovered_cover_img[r, c] -= shift_direction
    
    # 3. Format extracted msg text
    # 3.1. Find index of the last bit 1 of extracted msg str,
    # create a new string with content similar to the old string
    # but eliminate the last bits '1000 ...' based on the index of the last bit 1
    index = len(extracted_msg_str) - 1     
    while index >= 0:
        if extracted_msg_str[index] == '1':
            break
        index -= 1
    extracted_msg_str = extracted_msg_str[0:index]
    
    # 3.2. Convert extracted msg bit
    extracted_msg_bits = bitarray.bitarray(extracted_msg_str)
    extracted_msg_text = extracted_msg_bits.tobytes()
    
    # 4. Write the results to file
    # 4.1. Write extracted msg text to file
    f = open(extracted_msg_file, 'w')
    f.write(extracted_msg_text)
    f.close()
    
    # 4.2. Write recovered cover img to file
    imsave(recovered_cover_img_file, np.asarray(recovered_cover_img).astype('uint8'))
    
    return

## 5. Chạy thí nghiệm

Bạn sẽ gọi hàm nhúng và hàm rút trích ở trên với file cover `cover.bmp` và file message `msg.txt` mà mình đính kèm. Để minh chứng hàm nhúng và hàm rút trích đã được cài đặt đúng, bạn cần mở file message ban đầu và file message được rút trích, và in nội dung của hai file này ra; ngoài ra, bạn cũng cần đọc ảnh cover ban đầu và ảnh cover khôi phục được, và so sánh hai mảng ứng với hai ảnh này với nhau (dùng hàm `np.array_equal`)

In [4]:
# TODO
input_1 = 'cover.bmp'
input_2 = 'msg.txt'
output = 'stego.bmp'

t = embed_hist(input_1, input_2, output)
if (t == (-1, -1)):
    print 'An du lieu that bai'
else:
    output_1 = 'extracted_msg.txt'
    output_2 = 'recovered.bmp'
    extract_hist(output, t[0], t[1], output_1, output_2)
    
    f = open(input_2, 'r')
    msg_text = f.read()
    f.close()
    
    f = open(output_1, 'r')
    extracted_msg_text = f.read()
    f.close()
    
    cover_img = imread(input_1)
    recovered_img = imread(output_2)
    
    print msg_text
    print '-------------------------------'
    print extracted_msg_text
    print '-------------------------------'
    print np.array_equal(cover_img, recovered_img)

Empty your mind of all thoughts.
Let your heart be at peace.
-------------------------------
Empty your mind of all thoughts.
Let your heart be at peace.
-------------------------------
True


Theo bạn, trong trường hợp ảnh có nhiều peak point và zero point, ta nên chọn cặp peak point và zero point nào? Giải thích.

TODO

Trong trường hợp ảnh có nhiều peak point và zero point, ta nên chọn hai điểm peak point và zero point gần nhau nhất. Khi đó, phép dịch histogram sẽ thực hiện dịch trên ít giá trị nhất, do đó, ảnh sau ẩn dữ liệu sẽ ít biến đổi hơn (do những thay đổi của các điểm ảnh không nhiều) so với việc lựa chọn các điểm peak point và zero point khác.

---

Enjoy coding :-)